In [1]:
import pandas as pd
from preprocessing import resample_data, create_target
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("/home/kenny/algohub/data/data_eth_2018_2024.csv")

In [3]:
data['Time'] = pd.to_datetime(data['Time'])
data = data.set_index("Time")

In [4]:
time_frames = [15, 30, 60, 240, 1440]
new_data = resample_data(data, time_frames)

In [5]:
new_data.isna().sum().sum()

1785

In [6]:
new_data.dropna(inplace=True)

In [7]:
new_data.isna().sum().sum()

0

In [8]:
new_data = create_target(new_data, size=1500, future_time=10)

In [9]:
new_data["Target"] += 1

In [10]:
new_data["Target"].value_counts()

Target
1    342208
2    148280
0    146031
Name: count, dtype: int64

In [11]:
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, log_loss
from optuna.trial import Trial
import numpy as np
import optuna

In [12]:
begin_date = pd.to_datetime("2018-01-01T00:00:00")
end_date = pd.to_datetime("2023-01-01T00:00:00")

new_data_train = new_data[(new_data.index <= end_date)].copy()
new_data_test = new_data[(new_data.index > end_date)].copy()

In [13]:
features = list(set(new_data.columns) - set(["Target"]))

In [14]:
X_train = new_data_train[features]
y_train = new_data_train['Target']

X_test = new_data_test[features]
y_test = new_data_test['Target']

# model = lgb.train(params,
#                 lgb.Dataset(X_train, label=y_train),
#                 valid_sets=lgb.Dataset(X_test, label=y_test))

model = lgb.Booster(model_file='lgb_model.txt')

y_pred = model.predict(X_test)

In [15]:
y_pred

array([[0.3481336 , 0.32340632, 0.32846009],
       [0.32577833, 0.34773227, 0.3264894 ],
       [0.33506862, 0.32988816, 0.33504322],
       ...,
       [0.46896093, 0.09926931, 0.43176977],
       [0.46896093, 0.09926931, 0.43176977],
       [0.46896093, 0.09926931, 0.43176977]])

In [16]:
print(classification_report(y_test, np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.28      0.85      0.43     31242
           1       0.92      0.01      0.02     49326
           2       0.25      0.14      0.18     31945

    accuracy                           0.28    112513
   macro avg       0.48      0.34      0.21    112513
weighted avg       0.55      0.28      0.18    112513



In [17]:
new_data_test["Predictions"] = np.argmax(y_pred, axis=1)

In [18]:
begin_date = pd.to_datetime("2024-01-23T00:00:00")
template = new_data_test[new_data_test.index>=begin_date].copy()
template

,Open,High,Low,Close,Volume,Open_15min,High_15min,Low_15min,Close_15min,Volume_15min,...,Low_240min,Close_240min,Volume_240min,Open_1440min,High_1440min,Low_1440min,Close_1440min,Volume_1440min,Target,Predictions
Time,,,,,,,,,,,,,,,,,,,,,
2024-01-23 00:00:00,2314.19,2319.60,2312.67,2318.20,2111.3548,2314.69,2317.53,2310.21,2314.20,5510.5777,...,2303.59,2314.2,81266.0662,2457.06,2466.10,2303.59,2314.20,526337.6492,2,0
2024-01-23 00:05:00,2318.19,2324.65,2317.06,2324.38,4234.1497,2314.69,2317.53,2310.21,2314.20,5510.5777,...,2303.59,2314.2,81266.0662,2457.06,2466.10,2303.59,2314.20,526337.6492,2,0
2024-01-23 00:10:00,2324.38,2325.39,2319.03,2322.64,1272.6943,2314.69,2317.53,2310.21,2314.20,5510.5777,...,2303.59,2314.2,81266.0662,2457.06,2466.10,2303.59,2314.20,526337.6492,2,0
2024-01-23 00:15:00,2322.65,2327.11,2321.85,2326.12,1211.0385,2314.19,2325.39,2312.67,2322.64,7618.1988,...,2303.59,2314.2,81266.0662,2457.06,2466.10,2303.59,2314.20,526337.6492,1,0
2024-01-23 00:20:00,2326.11,2327.62,2324.46,2324.80,927.3362,2314.19,2325.39,2312.67,2322.64,7618.1988,...,2303.59,2314.2,81266.0662,2457.06,2466.10,2303.59,2314.20,526337.6492,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26 17:05:00,2279.86,2279.86,2275.00,2275.50,2430.7836,2270.88,2281.88,2269.11,2279.55,8583.4797,...,2234.46,2257.9,71448.4567,2235.02,2242.89,2171.30,2218.64,330426.6061,0,0
2024-01-26 17:10:00,2275.49,2276.69,2273.27,2273.68,2095.0957,2270.88,2281.88,2269.11,2279.55,8583.4797,...,2234.46,2257.9,71448.4567,2235.02,2242.89,2171.30,2218.64,330426.6061,0,0
2024-01-26 17:15:00,2273.69,2276.53,2271.82,2273.11,1503.6535,2279.55,2282.36,2273.27,2273.68,6643.3847,...,2234.46,2257.9,71448.4567,2235.02,2242.89,2171.30,2218.64,330426.6061,0,0


In [19]:
template["Predictions"] = np.argmax(model.predict(template[features]), axis=1)

In [21]:
from strategies import Simple
from backtesting import Backtest


artisanal_lightgbm = Backtest(new_data_test[features + ["Predictions"]], Simple, cash=10000, commission=.0004, margin=1)
output = artisanal_lightgbm.run()
output

Start                     2023-01-01 00:05:00
End                       2024-01-26 17:25:00
Duration                    390 days 17:20:00
Exposure Time [%]                    3.415605
Equity Final [$]                  9367.274388
Equity Peak [$]                       10000.0
Return [%]                          -6.327256
Buy & Hold Return [%]               89.724606
Return (Ann.) [%]                   -5.693861
Volatility (Ann.) [%]                1.654777
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -6.327256
Avg. Drawdown [%]                   -6.327256
Max. Drawdown Duration      390 days 17:10:00
Avg. Drawdown Duration      390 days 17:10:00
# Trades                                  569
Win Rate [%]                        26.537786
Best Trade [%]                       1.960784
Worst Trade [%]                     -1.091695
Avg. Trade [%]                    

In [23]:
artisanal_lightgbm.plot(resample=False)

GridPlot(id='p1350', ...)